In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import time
import math
import pandas as pd
import datetime
import os

In [ ]:
os.chdir('d:\\Users\\Patri\\Documents\\1.TCC\\Notebooks\\WebScrapAcordaosPDF')

In [ ]:
mes = "05"
mes_ano_inicial = mes + "/2020"
mes_ano_final = mes + "/2020"
mes_ano_inicial, mes_ano_final

In [ ]:
options = webdriver.ChromeOptions()
# options.add_argument("headless")
driver = webdriver.Chrome(executable_path=r"C:\chromedriver\chromedriver.exe", options=options)
driver.get("https://carf.fazenda.gov.br/sincon/public/pages/ConsultarJurisprudencia/consultarJurisprudenciaCarf.jsf")
driver.find_element(By.CSS_SELECTOR, ".container").click()
DataInicial = driver.find_element(By.ID, "dataInicialInputDate")
DataInicial.clear()
DataInicial.send_keys(mes_ano_inicial)
DataFinal = driver.find_element(By.ID, "dataFinalInputDate")
DataFinal.clear()
DataFinal.send_keys(mes_ano_final)
driver.find_element(By.ID, "botaoPesquisarCarf").click()
WebDriverWait(driver, 30000).until(EC.presence_of_element_located((By.ID, "j_id38")))

In [ ]:
contador = 1
quantidade_de_acordaos = int(driver.find_element(By.ID, "j_id38").text)
total_de_paginas = int(math.ceil(quantidade_de_acordaos/10))

print("Acórdão Inicial: " + str(contador))
print("Acórdão Final: " + str(quantidade_de_acordaos))
print("Total de páginas: " + str(total_de_paginas))

In [ ]:
# Criar Dataframe no Pandas com os dados do Acórdão
df = pd.DataFrame(columns = [
    "numProcesso",
    "contribuinte",
    "tipoRecurso", 
    "dataSessao",
    "relator",
    "numDecisao",
    "textDecisao",
    "ementa"
])

In [ ]:
df

In [ ]:
from datetime import datetime

inicio = datetime.now()
hora_inicio = inicio.strftime("%H:%M:%S")
inicio_string = inicio.strftime("%Y%m%B")
extension = ".txt"

file_name =  inicio_string + extension
file_log = open(file_name, 'w')
file_log.write(inicio_string)
file_log.close()

file_log = open(file_name, "a")
        
for pagina in range(0, total_de_paginas):
    if pagina == total_de_paginas-1:
        acordaos_da_pagina = (quantidade_de_acordaos - (total_de_paginas * 10)) % 10
        if acordaos_da_pagina == 0 : acordaos_da_pagina = 10
    else:
        acordaos_da_pagina = 10
    
    for item in range(0, acordaos_da_pagina):       
        decorrido = datetime.now() - inicio
                
        # Definir nome do elemento a clicar para pra abrir o detalhamento:
        contaLink = "tblJurisprudencia:" + str(int(contador)-1) + ":numDecisao"
        WebDriverWait(driver, 30000).until(EC.presence_of_element_located((By.ID, contaLink)))
        
        
        # Clicar no número do Acórdão pra abrir o detalhamento e download do PDF:
        driver.find_element(By.ID, contaLink).click()  
        
        # Capturar dados para o dataset
        numProcesso=driver.find_element(By.ID, "formAcordaos:numProcesso").text
        contribuinte=driver.find_element(By.ID, "formAcordaos:contribuinte").text
        tipoRecurso=driver.find_element_by_xpath("//div[@id='formAcordaos:tdivTipoRecurso']/span[2]").text
        dataSessao=driver.find_element(By.ID, "formAcordaos:dataSessao").text
        relator=driver.find_element(By.ID, "formAcordaos:relator").text
        numDecisao=driver.find_element(By.ID, "formAcordaos:numDecisao").text
        textDecisao=driver.find_element(By.ID, "formAcordaos:textDecisao").text
        if len(driver.find_elements(By.ID, "formAcordaos:ementa"))>0: # Checar se há ementa
            ementa=driver.find_element(By.ID, "formAcordaos:ementa").text
        else:
            ementa=""        

        # Enviar informações de Log
        expectativa = decorrido * (int(quantidade_de_acordaos) / int(contador) )       
        if decorrido > expectativa: 
            restante = 0 
        else: 
            restante = expectativa - decorrido
        log_line= str(decorrido)[:-7] + " " + str(expectativa)[:-7] + " " + str(restante)[:-7] + " " + str(numDecisao) + " " + str(numProcesso) + " " + contaLink + " " + str(pagina+1) + "/" + str(total_de_paginas) + "p " + str(contador) + "/" + str(quantidade_de_acordaos) + "a"
        print(log_line)
        file_log.write(log_line+"\n")           
            
        # Inserir dados do Acórdão no Dataframe
        df = df.append({ 
            "numProcesso"  : str(numProcesso), 
            "contribuinte" : contribuinte,
            "tipoRecurso"  : tipoRecurso, 
            "dataSessao"   : str(dataSessao),
            "relator"      : relator,
            "numDecisao"   : str(numDecisao),
            "textDecisao"  : textDecisao,
            "ementa"       : ementa
        }, ignore_index=True)
      
        #   Clicar no botão de download do PDF:
        WebDriverWait(driver, 30000).until(EC.presence_of_element_located((By.XPATH, "//div/a/img")))
        driver.find_element(By.XPATH, "//div/a/img").click()  
        contador += 1

        #   Clicar no botão de Voltar:  
        driver.find_element(By.XPATH, "//div[3]/input").click()      

    #   Clicar no botão de Próxima Página:  
    driver.find_element(By.XPATH, "//table[@id='dataScroller_1_table']/tbody/tr/td[5]/div").click()           
    
file_log.close()

# Salvar DataFrame
nomeArquivoCSV = "Acordaos_" + re.sub("/","",mes_ano_inicial) + "a" + re.sub("/","",mes_ano_final) + "_total_" + str(quantidade_de_acordaos)
df.to_csv(nomeArquivoCSV + ".csv")

nomeArquivoCSV

In [ ]:
# Salvar DataFrame
nomeArquivoCSV = "Acordaos_" + re.sub("/","",mes_ano_inicial) + "a" + re.sub("/","",mes_ano_final) + "_total_" + str(quantidade_de_acordaos)
df.to_csv(nomeArquivoCSV + ".csv")

nomeArquivoCSV

In [ ]:
os.getcwd()